In [ ]:
import sys
import os
script_dir = os.getcwd()
root_dir = os.path.join(os.path.dirname(os.path.abspath(script_dir)))
sys.path.append(root_dir)
os.chdir(root_dir)

import json
import pandas as pd
import numpy as np
from utils.io import load_object, save_object
import requests
from collections import Counter
import spacy
import faiss
from openai import OpenAI
from utils.embedding import index_context_db
from dotenv import load_dotenv
import random
import math
from utils.check_db_version import get_local_version


In [ ]:
_VERSION=get_local_version()
_VERSION

'2025-10-03'

## Create synthetic queries and answers

In [42]:
with open(f"data/latest_db/fda_statements__{_VERSION}.json", "r") as f:
    fda_statements = json.load(f)

In [43]:
import ast
standardized_to_raw_mapping=pd.read_csv(f"data/latest_db/moalmanac_fda_core__{_VERSION}.csv")
print(standardized_to_raw_mapping.shape[0])
if standardized_to_raw_mapping.shape[0]==standardized_to_raw_mapping.drop_duplicates().shape[0]:
    print("No duplicate rows")

651
No duplicate rows


In [44]:
standardized_to_raw_mapping['biomarker']=standardized_to_raw_mapping['biomarker'].apply(ast.literal_eval)
standardized_to_raw_mapping['therapy']=standardized_to_raw_mapping['therapy'].apply(ast.literal_eval)
standardized_to_raw_mapping.head()

,statement_id,standardized_cancer,raw_cancer,modified_standardized_cancer,biomarker,therapy
0,0,invasive breast carcinoma,early breast cancer,early invasive breast carcinoma,"[ER positive, HER2-negative]","[Tamoxifen, Abemaciclib]"
1,1,invasive breast carcinoma,early breast cancer,early invasive breast carcinoma,"[PR positive, HER2-negative]","[Tamoxifen, Abemaciclib]"
2,2,invasive breast carcinoma,early breast cancer,early invasive breast carcinoma,"[PR positive, ER positive, HER2-negative]","[Tamoxifen, Abemaciclib]"
3,3,invasive breast carcinoma,advanced or metastatic breast cancer,advanced or metastatic invasive breast carcinoma,"[ER positive, HER2-negative]","[Anastrozole, Abemaciclib]"
4,4,invasive breast carcinoma,advanced or metastatic breast cancer,advanced or metastatic invasive breast carcinoma,"[PR positive, HER2-negative]","[Anastrozole, Abemaciclib]"


In [53]:
from utils.flatten_statement import flatten_statements

def create_synthetic_query(db: dict) -> str:
    """
    Create a synthetic query based on the database entry.
    """
    if len(db['biomarker']) > 1:
        biomarker_str = ", ".join(db['biomarker'])
    else:
        biomarker_str = db['biomarker'][0]
    query_text=f"if a patient with {db['modified_standardized_cancer']} cancer has {biomarker_str.lower()}, what therapy is recommended?"
    return query_text

def create_synthetic_answer(db: dict) -> str:
    """
    Create a synthetic answer based on the database entry.
    """
    if len(db['biomarker']) > 1:
        biomarker_str = ", ".join(db['biomarker'])
    else:
        biomarker_str = db['biomarker'][0]
    if len(db['therapy']) > 1:
        therapy_str = " + ".join(db['therapy'])
    else:
        therapy_str = db['therapy'][0]
    answer_text=f"if a patient with {db['modified_standardized_cancer']} cancer has {biomarker_str.lower()}, one recommended therapy is {therapy_str.lower()}."
    return answer_text

synthetic_query_list = []
synthetic_answer_list = []
synthetic_query_therapy_pair_dict = {}
for _, row in standardized_to_raw_mapping.iterrows():
    #create synthetic query and answer and add to dataframe
    query = create_synthetic_query(row)
    answer = create_synthetic_answer(row)
    standardized_to_raw_mapping.at[_, 'prompt'] = query
    
    #add more therapy context to answers (context db)
    summary, stmt_row = flatten_statements(fda_statements[_])
    if len(stmt_row['therapy_type']) > 1:
        therapy_type = ' + '.join(stmt_row['therapy_type'])
    else:
        therapy_type = stmt_row['therapy_type'][0]
    if len(stmt_row['therapy_strategy']) > 1:
        therapy_strategy = ' + '.join(stmt_row['therapy_strategy'])
    else:
        therapy_strategy = stmt_row['therapy_strategy'][0]
        
    standardized_to_raw_mapping.at[_, 'answer'] = (
        f"{answer} therapy type: {therapy_type.lower()}. therapy strategy: {therapy_strategy.lower()}. indication: {stmt_row['indication'].lower()} approval url: {stmt_row['approval_url']}"
    )
    
    #create synthetic query and ground-truth mapping dict
    row['therapy'] = [therapy.lower().strip() for therapy in row['therapy']]
    if query in synthetic_query_therapy_pair_dict.keys():
        synthetic_query_therapy_pair_dict[query].append(set(row['therapy']))
    else:
        synthetic_query_therapy_pair_dict[query] = [set(row['therapy'])]

In [46]:
#split ranges for testing on small batches
def split_ranges(max_int, num_ranges, samples=None, seed=42):
    random.seed(seed)
    step = math.ceil(max_int / num_ranges)
    ranges = [(i*step+1, min((i+1)*step, max_int)) for i in range(num_ranges)]
    
    if samples is None:
        return ranges
    
    # split samples evenly across ranges
    n_per_range = samples // num_ranges
    remainder = samples % num_ranges
    
    result = []
    for i, r in enumerate(ranges):
        count = n_per_range + (1 if i < remainder else 0)
        result.extend(random.randint(r[0], r[1]) for _ in range(count))
    
    return result

test_idx_list = split_ranges(standardized_to_raw_mapping.shape[0], 10, 50)

In [55]:
for i, e in enumerate(standardized_to_raw_mapping.loc[test_idx_list].prompt):
    print(i, e)
    if i == 10:
        break

0 if a patient with metastatic castration-resistant prostate adenocarcinoma cancer has brca1 oncogenic variants, what therapy is recommended?
1 if a patient with advanced or metastatic invasive breast carcinoma cancer has pr positive, her2-negative, what therapy is recommended?
2 if a patient with metastatic non-small cell lung cancer cancer has pd-l1 >= 10% tiic, what therapy is recommended?
3 if a patient with philadelphia chromosome-positive chronic myelogenous leukemia cancer has bcr::abl1, what therapy is recommended?
4 if a patient with locally advanced or metastatic non-small cell lung cancer cancer has egfr exon 20 (insertion), what therapy is recommended?
5 if a patient with metastatic colorectal adenocarcinoma cancer has egfr positive, wild type kras, what therapy is recommended?
6 if a patient with non-small cell lung cancer cancer has wild type ros1, wild type egfr, wild type alk, what therapy is recommended?
7 if a patient with metastatic non-small cell lung cancer cancer 

In [56]:
for i, e in enumerate(standardized_to_raw_mapping.loc[test_idx_list].answer):
    print(i, e)
    if i == 10:
        break

0 if a patient with metastatic castration-resistant prostate adenocarcinoma cancer has brca1 oncogenic variants, one recommended therapy is prednisone + abiraterone acetate + niraparib. therapy type: chemotherapy + hormone therapy + targeted therapy. therapy strategy: corticosteroid + antiandrogen + parp inhibition. indication: akeega is a combination of niraparib, a poly (adp-ribose) polymerase (parp) inhibitor, and abiraterone acetate, a cyp17 inhibitor indicated with prednisone for the treatment of adult patients with deleterious or suspected deleterious brca-mutated (brcam) metastatic castration-resistant prostate cancer (mcrpc). select patients for therapy based on an fda-approved test for akeega. approval url: https://www.accessdata.fda.gov/drugsatfda_docs/label/2023/216793s000lbl.pdf
1 if a patient with advanced or metastatic invasive breast carcinoma cancer has pr positive, her2-negative, one recommended therapy is anastrozole + abemaciclib. therapy type: hormone therapy + targ

In [57]:
synthetic_query_therapy_pair_dict

{'if a patient with early invasive breast carcinoma cancer has er positive, her2-negative, what therapy is recommended?': [{'abemaciclib',
   'tamoxifen'},
  {'abemaciclib', 'letrozole'},
  {'anastrozole', 'goserelin', 'ribociclib'},
  {'goserelin', 'letrozole', 'ribociclib'},
  {'abemaciclib', 'anastrozole'},
  {'abemaciclib', 'exemestane'}],
 'if a patient with early invasive breast carcinoma cancer has pr positive, her2-negative, what therapy is recommended?': [{'abemaciclib',
   'tamoxifen'},
  {'anastrozole', 'goserelin', 'ribociclib'},
  {'goserelin', 'letrozole', 'ribociclib'},
  {'abemaciclib', 'letrozole'},
  {'abemaciclib', 'anastrozole'},
  {'abemaciclib', 'exemestane'}],
 'if a patient with early invasive breast carcinoma cancer has pr positive, er positive, her2-negative, what therapy is recommended?': [{'abemaciclib',
   'tamoxifen'},
  {'abemaciclib', 'letrozole'},
  {'anastrozole', 'goserelin', 'ribociclib'},
  {'goserelin', 'letrozole', 'ribociclib'},
  {'abemaciclib',

In [50]:
save_object(synthetic_query_therapy_pair_dict, "data/latest_db/synthetic_query_therapy_pair_dict.pkl")

In [58]:
for k, v in synthetic_query_therapy_pair_dict.items():
    print(k, v)
    break

if a patient with early invasive breast carcinoma cancer has er positive, her2-negative, what therapy is recommended? [{'abemaciclib', 'tamoxifen'}, {'abemaciclib', 'letrozole'}, {'anastrozole', 'ribociclib', 'goserelin'}, {'ribociclib', 'goserelin', 'letrozole'}, {'anastrozole', 'abemaciclib'}, {'abemaciclib', 'exemestane'}]


In [ ]:
standardized_to_raw_mapping.to_csv(f"data/latest_db/moalmanac_fda_core_query__{_VERSION}.csv")

### Save synthetic prompts and answers/context

In [52]:
with open(f"data/latest_db/synthetic_prompts__{_VERSION}.json", "w") as f:
    json.dump(standardized_to_raw_mapping.prompt.to_list(), f)

with open(f"data/latest_db/synthetic_answers__{_VERSION}.json", "w") as f:
    json.dump(standardized_to_raw_mapping.answer.to_list(), f)    

### Save synthetic answers as context index

In [37]:
_CONTEXT = standardized_to_raw_mapping.answer.to_list()
print(_CONTEXT[0])
print(len(_CONTEXT))

If a patient with early invasive breast carcinoma cancer has her2-negative, er positive, one recommended therapy is tamoxifen + abemaciclib. therapy type: hormone therapy + targeted therapy. therapy strategy: estrogen receptor inhibition + cdk4/6 inhibition. indication: verzenio is a kinase inhibitor indicated in combination with endocrine therapy (tamoxifen or an aromatase inhibitor) for the adjuvant treatment of adult patients with hormone receptor (hr)-positive, human epidermal growth factor receptor 2 (her2)-negative, node positive, early breast cancer at high risk of recurrence. approval url: https://www.accessdata.fda.gov/drugsatfda_docs/label/2023/208716s010s011lbl.pdf
642


## Create validation test dataset

In [59]:
standardized_to_raw_mapping_subset=standardized_to_raw_mapping.sample(frac=.1, random_state=42)
standardized_to_raw_mapping_subset

,statement_id,standardized_cancer,raw_cancer,modified_standardized_cancer,biomarker,therapy,prompt,answer
86,87,Colorectal Adenocarcinoma,metastatic colorectal cancer,metastatic colorectal adenocarcinoma,[BRAF p.V600E],"[Cetuximab, Encorafenib]",if a patient with metastatic colorectal adenoc...,If a patient with metastatic colorectal adenoc...
192,193,Acute Lymphoid Leukemia,B-cell precursor acute lymphoblastic leukemia ...,b-cell precursor acute lymphoid leukemia,[CD22 +],[Inotuzumab ozogamicin],if a patient with b-cell precursor acute lymph...,If a patient with b-cell precursor acute lymph...
479,481,Non-Small Cell Lung Cancer,locally advanced or metastatic non-small cell ...,locally advanced or metastatic non-small cell ...,[KRAS p.G12C],[Sotorasib],if a patient with locally advanced or metastat...,If a patient with locally advanced or metastat...
616,619,Invasive Breast Carcinoma,early breast cancer,early invasive breast carcinoma,"[HER2-negative, ER positive]","[Anastrozole, Abemaciclib]",if a patient with early invasive breast carcin...,If a patient with early invasive breast carcin...
72,73,Invasive Breast Carcinoma,locally advanced or metastatic breast cancer,locally advanced or metastatic invasive breast...,"[PTEN splice site variants, HER2-negative, ER ...","[Fulvestrant, Capivasertib]",if a patient with locally advanced or metastat...,If a patient with locally advanced or metastat...
...,...,...,...,...,...,...,...,...
165,166,Invasive Breast Carcinoma,advanced or metastatic breast cancer,advanced or metastatic invasive breast carcinoma,"[PR positive, HER2-negative, ER positive]","[Fulvestrant, Abemaciclib]",if a patient with advanced or metastatic invas...,If a patient with advanced or metastatic invas...
78,79,Non-Small Cell Lung Cancer,metastatic non-small cell lung cancer,metastatic non-small cell lung cancer,[MET Exon 14 (Splice Site)],[Capmatinib],if a patient with metastatic non-small cell lu...,If a patient with metastatic non-small cell lu...
181,182,Chronic Myelogenous Leukemia,philadelphia chromosome-positive chronic myelo...,philadelphia chromosome-positive chronic myelo...,[BCR::ABL1],[Imatinib],if a patient with philadelphia chromosome-posi...,If a patient with philadelphia chromosome-posi...
587,589,Non-Small Cell Lung Cancer,non-small cell lung cancer,non-small cell lung cancer,"[Wild type EGFR, Wild type ALK]","[Carboplatin, Paclitaxel, Nivolumab, Ipilimumab]",if a patient with non-small cell lung cancer c...,If a patient with non-small cell lung cancer c...


In [60]:
standardized_to_raw_mapping_subset.to_csv("data/latest_db/moalmanac_small_testing_dataset.csv", index=False)